## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization

from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170508288/170498071 [==============================] - 7s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu',padding='same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
classifier.add(BatchNormalization())
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size =(2,2)))
classifier.add(Dropout(0.2))

#卷積組合
classifier.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
classifier.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size =(2,2)))
classifier.add(Dropout(0.2))

classifier.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
classifier.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size =(2,2)))
classifier.add(Dropout(0.2))



#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128,activation='relu')) #
classifier.add(Dropout(0.2))
#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=10, batch_size=100, verbose=1)

2022-05-23 08:13:16.614892: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-05-23 08:13:18.085111: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
500/500 [==============================] - 128s 253ms/step - loss: 1.5582 - accuracy: 0.4411
Epoch 2/10
500/500 [==============================] - 126s 252ms/step - loss: 1.0969 - accuracy: 0.6096
Epoch 3/10
500/500 [==============================] - 128s 255ms/step - loss: 0.8948 - accuracy: 0.6860
Epoch 4/10
500/500 [==============================] - 128s 256ms/step - loss: 0.7743 - accuracy: 0.7329
Epoch 5/10
500/500 [==============================] - 128s 255ms/step - loss: 0.6842 - accuracy: 0.7622
Epoch 6/10
500/500 [==============================] - 129s 257ms/step - loss: 0.6174 - accuracy: 0.7871
Epoch 7/10
500/500 [==============================] - 128s 256ms/step - loss: 0.5641 - accuracy: 0.8049
Epoch 8/10
500/500 [==============================] - 128s 257ms/step - loss: 0.5168 - accuracy: 0.8206
Epoch 9/10
500/500 [==============================] - 131s 261ms/step - loss: 0.4787 - accuracy: 0.8328
Epoch 10/10
500/500 [==============================] - 131s 262m

In [5]:
print(classifier.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        3

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[3.8446716e-04, 5.6799322e-06, 4.6570422e-03, 9.0540421e-01,
        8.0583245e-02, 1.7677125e-03, 1.2051166e-03, 1.7416473e-04,
        5.5595781e-03, 2.5877121e-04]], dtype=float32)